In [ ]:
import rioxarray

#See docs for rioxarray.open_rasterio

rds = rioxarray.open_rasterio('./r50.0_tile7.tif')
rds

In [ ]:
import hvplot.xarray
rds.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='viridis', clim=(1,255))

In [ ]:
s3_file_obj = 's3://ws-enduser/USA/r50.0_tile7/2020/etasw_202005.tif'

rds1 = rioxarray.open_rasterio(s3_file_obj)


In [ ]:
rds1.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='viridis', clim=(1,255))

https://www.geeksforgeeks.org/find-the-nearest-value-and-the-index-of-numpy-array/

In [ ]:
rds = rioxarray.open_rasterio('./r50.0_tile7.tif')
my_arr = rds.values

In [ ]:
my_arr.shape

In [ ]:
yheight = my_arr.shape[1]

In [ ]:
xwidth = my_arr.shape[2]

In [ ]:
! rio info ./r50.0_tile7.tif

In [ ]:
import numpy as np
iam_broke = False
for y in range(0,yheight):
    for x in range(0,xwidth):
        if np.isnan(my_arr[0][y,x]):
            print(y,x)
            y_index = y
            x_index = x
            iam_broke = True
            break
    if iam_broke:
        break
    

In [ ]:
for x in range(x_index,x_index+1220):
    for y in range(y_index,y_index+1230):
        my_arr[0][y,x] = 250
        
rds[:] = my_arr

rds.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='viridis', clim=(1,255))

In [ ]:
for x in range(x_index,x_index+1220):
    for y in range(y_index,y_index+1230):
        my_arr[0][y,x] = int(y/250*25)
        
rds[:] = my_arr

rds.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='viridis', clim=(1,255))

In [ ]:
rds1.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='viridis', clim=(1,255))

In [ ]:
best_arr = rds1.values

In [ ]:
for x in range(x_index,x_index+1220):
    for y in range(y_index,y_index+1230):
        my_arr[0][y,x] = best_arr[0][y,x]
        
rds[:] = my_arr

rds.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='viridis', clim=(1,255))

In [ ]:
!aws s3 ls ws-out/good/

In [ ]:
bigds = rioxarray.open_rasterio('s3://ws-out/good/etasw_202005.tif')

In [ ]:
bigds

In [ ]:
big_arr = bigds.values

In [ ]:
yheight = big_arr.shape[1]
yheight

In [ ]:
xwidth = big_arr.shape[2]
xwidth

In [ ]:
import numpy as np
iam_broke = False
for y in range(0,yheight):
    for x in range(0,xwidth):
        if np.isnan(big_arr[0][y,x]):
            print(y,x)
            by_index = y
            bx_index = x
            iam_broke = True
            break
    if iam_broke:
        break

In [ ]:
!aws s3 cp s3://ws-out/good/gridmeister.py .

In [ ]:
import sys
sys.path.append('.')

In [ ]:
! cat gridmeister.py

In [ ]:
! ls tile_aoi

In [ ]:
tile_name = 'r50.0_tile7'
geojson_file = f'./tile_aoi/{tile_name}.geojson'

In [ ]:
from gridmeister import GridMeisterTile

In [ ]:
import geopandas as gpd
def return_geojson_extent(geojson_file):
    my_geo = gpd.read_file(geojson_file)
    smallest_extent = my_geo.geometry[0].bounds
    return(smallest_extent)
    pass


In [ ]:
smallest_extent = return_geojson_extent(geojson_file)

In [ ]:
smallest_extent

In [ ]:
divisor=5
tile='./chips/' + tile_name
gm = GridMeisterTile(tile, smallest_extent, divisor)

In [ ]:
lst = gm.chip_list()

In [ ]:
! mkdir -p ./AOI

In [ ]:
for l in lst:
    gm.create_chip_shp(l)

In [ ]:
! ls ./AOI

In [ ]:
my_geo = './AOI/chip42.2N-93.8E.json'

In [ ]:
rds1.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='viridis', clim=(1,255))

In [ ]:
from shapely.geometry import mapping

def make_geom(aoi_filename):
    '''creates a geom/Polygon/Coordinates - bbox - from an geojson file'''
    # read in AOI as a GeoDataFrame
    aoi = gpd.read_file(aoi_filename)

    # get the geometry of the AOI as a dictionary for use with PySTAC Client
    geom = mapping(aoi.to_dict()['geometry'][0])
    return geom

In [ ]:
geom=make_geom(my_geo)

In [ ]:
geom

In [ ]:
my_ds = rds1.rio.clip([geom], crs='epsg:4326')

In [ ]:
my_ds.hvplot.image(rasterize=True,  x='x', y='y', width=500, height=400, cmap='viridis', clim=(1,255))

In [ ]:
my_patch_arr = my_ds.values

In [ ]:
my_patch_arr.shape

In [ ]:
patch_yheight=my_patch_arr.shape[1]
patch_xwidth=my_patch_arr.shape[2]

In [ ]:
bigds.x

In [ ]:
bigds.y

In [ ]:
import numpy as np

def get_xy_index(xfind, yfind, xlons, ylats):
    arr = xlons
    print("Array is : ", arr)

    # element to which nearest value is to be found
    x = xfind
    print("Value to which nearest element is to be found: ", x)

    # calculate the difference array
    difference_array = np.absolute(arr-x)

    # find the index of minimum element from the array
    index = difference_array.argmin()
    print("Nearest element to the given values is : ", arr[index])
    print("Index of nearest value is : ", index)
    x_index = index
    
    arr = ylats
    print("Array is : ", arr)
  
    # element to which nearest value is to be found
    x = yfind
    print("Value to which nearest element is to be found: ", x)

    # calculate the difference array
    difference_array = np.absolute(arr-x)

    # find the index of minimum element from the array
    index = difference_array.argmin()
    print("Nearest element to the given values is : ", arr[index])
    print("Index of nearest value is : ", index)
    y_index = index

    return(x_index, y_index)


In [ ]:
xfind=-91.262
yfind=39.74
xlons = bigds.x.values
ylats = bigds.y.values
x_index,y_index = get_xy_index(xfind, yfind, xlons, ylats)

In [ ]:
geom['coordinates']

In [ ]:
ll = geom['coordinates'][0][3]

In [ ]:
xfind = ll[0]
xfind

In [ ]:
yfind = ll[1]
yfind

In [ ]:
xlons = bigds.x.values
ylats = bigds.y.values
x_index,y_index = get_xy_index(xfind, yfind, xlons, ylats)

In [ ]:
x_index,y_index

In [ ]:
yp=0
for y in range(y_index,y_index+patch_yheight):
    xp=0
    for x in range(x_index,x_index+patch_xwidth):
        my_yp = y-y_index
        my_xp = x-x_index
        #big_arr[0][y,x] = my_patch_arr[0][yp,xp]
        big_arr[0][y,x] = my_patch_arr[0][my_yp,my_xp]
        #big_arr[0][y,x] = 88 
        #print(xp)
        xp=xp+1
    yp+yp+1
        
bigds[:] = big_arr

subset1 = bigds.isel(x=slice(0,49000,10), y=slice(0,25000,10))
bigds.hvplot.image(rasterize=True,  x='x', y='y', width=900, height=400, cmap='viridis', clim=(1,255))

In [ ]:
my_patch_arr